### Connect to mongodb and make fure we have access to the pdf files

In [1]:
from pymongo import MongoClient
from gridfs import GridFS

# Connect to MongoDB
db = MongoClient('mongodb://10.49.0.6:27017')['supercias']
companies = db['companies']

#find one
comp = companies.find_one()
#print(comp['Documentos online'])
pdfs = []
#if company has documents
for value in comp['Documentos online'].values():
    if isinstance(value, list) and len(value) > 0:
        [ pdfs.append(v) for v in value ]

print(pdfs)



['1000000000008_DocumentosJuridicos_CANCELACIÓN_Resolución_2007-01-03.pdf']


### make a query to get the pdfs we are interested in

In [2]:
# Access the GridFS collection
fs = GridFS(db, collection='companies')

# Retrieve all files from GridFS collection that have in the filen ethe substring 'Balance  Estado de Situación'
pdf_queries = fs.find({'filename': {'$regex': '.*Balance  Estado de Situación Financiera_2022.*'}})

# print how many files were found
# this will exahust the cursor
#print('found:', len(list(files)))




### Write queried pdfs to file system

In [10]:
# Write the contents of the file to a new file
number_of_pdf_to_download = 100
counter = 0
path = '../../storage/pdfs/'
for pdf_query in pdf_queries:
    if counter >= number_of_pdf_to_download:
        break
    buffer = pdf_query.read()
    if buffer.startswith(b'%PDF'):
        # write the file to the disk
        with open( path + pdf_query.filename, 'wb') as f:
            f.write(buffer)
            counter += 1

### Parse data from pdf

In [53]:
# get every file in the path directory
import os

from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer 

def extract_text_from_pdf(pdf_path):
    try:
        lines = []
        for page_layout in extract_pages(pdf_path):
            same_y_axis = {}
            # get all the elements that are on the same y0 position
            for element in page_layout:
                if isinstance(element, LTTextContainer):
                    if element.y0 not in same_y_axis:
                        same_y_axis[element.y0] = []
                    same_y_axis[element.y0].append(element)
            # concat the lines
            for y0, elements in same_y_axis.items():
                line = []
                for element in elements:
                    text = element.get_text()
                    # sanitize the text
                    text = text.replace('\n', ' ')
                    text = text.replace('\t', ' ')
                    text = text.strip()
                    line.append({ 'y': y0, 'text': text })
                lines.append(line)
        # parse the lines
        data = {}
        # filter only the ones that have 2 elements
        reachedEndOfHeader = False
        reachedStartOfFooter = False
        for line in lines:
            if not reachedEndOfHeader:
                if(len(line) == 1):
                    if('ESTADO DE SITUACIÓN FINANCIERA' in line[0]['text']):
                        reachedEndOfHeader = True
                else:
                    data[line[0]['text']] = line[1]['text']
            elif reachedEndOfHeader and not reachedStartOfFooter:
                if('ESTADO DE SITUACIÓN FINANCIERA' in line[0]['text']):
                    continue
                elif('CUENTA'in line[0]['text'] and 'CÓDIGO' in line[1]['text']):
                    continue
                else:
                    data[line[0]['text']] = {}
                    if(len(line) >= 2):
                        data[line[0]['text']]['codigo'] = line[1]['text']
                    if(len(line) == 3):
                        data[line[0]['text']]['descripcion'] = line[2]['text']
                if('REPRESENTANTE(S) LEGAL(ES)' in line[0]['text']):
                    reachedStartOfFooter = True
            elif reachedStartOfFooter:
                # need to implement later
                continue
    except Exception as e:
        print('Error:', e)
        data = {}
    
    return data


files = os.listdir(path)
# for file in files:
file = files[8]
# openfile and read it
text = extract_text_from_pdf(path + file)
# print(text)
print(text)

[{'y': 208.451, 'text': 'PAUTA GAVILANES EDISON GERARDO'}]
[{'y': 193.47099999999998, 'text': '0102931763'}]
[{'y': 210.451, 'text': 'null'}]
[{'y': 195.47099999999998, 'text': 'null'}]
[{'y': 180.47099999999998, 'text': 'null'}]
[{'y': 97.401, 'text': 'EL REPRESENTANTE LEGAL DECLARA QUE LOS DATOS QUE CONSTAN EN ESTOS ESTADOS FINANCIEROS SON EXACTOS Y VEREDEROS. LOS ESTADOS FINANCIEROS ESTÁN ELAORADOS BAJO LAS NORMAS INTERNACIONALES DE INFORMACIÓN FINANCIERA.'}]
[{'y': 79.95100000000001, 'text': 'SUPERINTENDENCIA DE COMPAÑÍAS, VALORES Y SEGUROS'}]
[{'y': 57.401, 'text': 'CERTIFICO Que el presente balance ha sido enviado electrónicamente por el Representante Legal en virtud de una Declaración de Responsabilidad firmada por éste que obvia las firmas autógrafas. Esta copia es fiel reproducción del documento que consta en nuestros'}]
{'RAZÓN SOCIAL': 'MANUEL MENDEZ CONSTRUCTORA MMCONS CIA.LTDA.', 'DIRECCIÓN': 'AV. ORDOÑEZ LASSO Y BUGANVILLA No. 1324 BARRIO: SECTOR VIRGEN DEL MILAGRO', 'EXP

### extract the text from the pdf

In [20]:
from pdf2image import convert_from_path

pdfs_images = []
# for every filename pdf
for filename in filenames:
    try:
        # Convert PDF to images
        images = convert_from_path(filename)

        # Save each image as a separate file
        for i, image in enumerate(images):
            pdfs_images.append(f'{filename}_{i}.jpg')
            image.save(f'{filename}_{i}.jpg', 'JPEG')
    except Exception as e:
        print(e)
        print(f'Error processing {filename}')
        continue




Unable to get page count.
Syntax Warning: May not be a PDF file (continuing anyway)
Syntax Error (2): Illegal character <48> in hex string
Syntax Error (3): Illegal character <54> in hex string
Syntax Error (4): Illegal character <4d> in hex string
Syntax Error (5): Illegal character <4c> in hex string
Syntax Error (11): Illegal character <48> in hex string
Syntax Error (20): Illegal character <73> in hex string
Syntax Error (23): Illegal character <68> in hex string
Syntax Error (24): Illegal character <72> in hex string
Syntax Error (27): Illegal character <3d> in hex string
Syntax Error (28): Illegal character <22> in hex string
Syntax Error (29): Illegal character <2f> in hex string
Syntax Error (30): Illegal character <22> in hex string
Syntax Error (32): Illegal character <2f> in hex string
Syntax Error (36): Illegal character <73> in hex string
Syntax Error (38): Illegal character <72> in hex string
Syntax Error (39): Illegal character <69> in hex string
Syntax Error (40): Illeg

In [27]:
import easyocr

%pip install opencv-python-headless
reader = easyocr.Reader(['es'])
extract_info = reader.readtext('/home/terac/data-mining/supercias_second/src/clean/output_image_0.jpg')



Defaulting to user installation because normal site-packages is not writeable


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Note: you may need to restart the kernel to use updated packages.


In [28]:
extract_info = [ el[1] for el in extract_info ]
print(extract_info)

['REPUBLICA', 'DEL', 'ECUADOR', 'SUPERINTENDENCIA', 'DE COMPAÑIAS', '8 5', 'RESOLUCION No. 07.Q.IJ.000025', 'DRA. PIEDAD MONCAYO DE VASCONEZ', 'DIRECTORA', 'JURIDICA DEL DEPARTAMENTO', 'DE DISOLUCION Y LIQUIDACION DE COMPAÑIAS', 'co NSIDERA ND O:', 'QUE en virtud de lo dispuesto en el inciso segundo del artículo 405, de la', 'de', 'Compañías; el Superintendente de Compañías; sin ningún otro trámite', 'ordenar la', 'cancelación de la inscripción en el Registro Mercantil de las compañías cuyo trámite de', 'liquidación', 'no', 'hubiere  terminado', 'en', 'el', 'de', 'un', 'año  posterior', 'a', 'la fecha de', 'expedición de la Resolución de Disolución;', 'QUE por Resolución No.85-1-2-1-379 de 21 de', 'marzo de 1985, inscrita en el', 'Registro Mercantil del Distrito Metropolitano de Quito', 'el 6 de marzo', 'de 1986,', 'esta', 'Superintendencia', 'declaró', 'disuelta', 'a', 'la COMPAÑIA', 'AGRICOLA', 'Y PRODUCTORA', 'DE ALIMENTOS BALANCEADOS SOCIEDAD ANONIMA. (CAPRODAL SA), EN', 'LIQUIDACI